In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import sys
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
from tensorflow import keras
import os
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator



/burg/glab/users/sk4973/venv/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(
2021-09-09 17:44:05.848121: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2021-09-09 17:44:05.848205: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
UsageError: Line magic function `%tensorflow_version` not found.


In [2]:
sys.path.insert(0, '../src')

from utils import df_to_xarray,read_xarray,plot_image,preprocess_image

In [3]:
dir_name="../data/data1"
val_dir_name="../data/data2"

chl,mld,sss,sst,u10,fg_co2,xco2,icefrac,patm,pco2=read_xarray(dir_name)

/burg/glab/users/sk4973/venv/lib/python3.8/site-packages/xarray/backends/plugins.py:61: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [4]:
chl_images=preprocess_image(chl.Chl.data)
mld_images=preprocess_image(mld.MLD.data)
sss_images=preprocess_image(sss.SSS.data)
sst_images=preprocess_image(sst.SST.data)
xco2_images=preprocess_image(xco2.XCO2.data,xco2=True)
pco2_images=preprocess_image(pco2.pCO2.data,pco2=True)


In [8]:
xco2_images.shape, pco2_images.shape #dimensions

((421, 180, 360, 1), (421, 180, 360, 1))

In [9]:
pco2.pCO2.data.max(), pco2.pCO2.data.min(), pco2.pCO2.data.std() #pco2 information

(831.1136616134033, 0.0, 171.05473677144028)

In [11]:
X = np.stack((chl_images, mld_images, sss_images, sst_images,xco2_images), axis = 1)

In [12]:
INPUT_SHAPE=X[0].shape
OUTPUT_SHAPE=pco2_images[0].shape

In [17]:
from functools import partial
from tensorflow import keras

DefaultConv3D = partial(keras.layers.Conv3D,
                        kernel_size=3,activation='relu', padding="SAME")

base_model = keras.models.Sequential([
    DefaultConv3D(filters=64, input_shape=INPUT_SHAPE),
    DefaultConv3D(filters=64),
    keras.layers.MaxPooling3D(pool_size=3),
    DefaultConv3D(filters=128),
    DefaultConv3D(filters=128),
    keras.layers.Dropout(0.3),
    keras.layers.UpSampling3D(size=(1,3,3)),
    DefaultConv3D(filters=64),
    DefaultConv3D(filters=2),
    DefaultConv3D(filters=1,kernel_size=1),
    keras.layers.Reshape(OUTPUT_SHAPE)
   
])

base_model.summary()

2021-09-09 17:52:31.162975: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2021-09-09 17:52:31.180523: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-09 17:52:31.187790: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (g008): /proc/driver/nvidia/version does not exist
2021-09-09 17:52:31.424652: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_2 (Conv3D)            (None, 5, 180, 360, 64)   1792      
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 5, 180, 360, 64)   110656    
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 1, 60, 120, 64)    0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 1, 60, 120, 128)   221312    
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 1, 60, 120, 128)   442496    
_________________________________________________________________
dropout (Dropout)            (None, 1, 60, 120, 128)   0         
_________________________________________________________________
up_sampling3d (UpSampling3D) (None, 1, 180, 360, 128)  0

In [18]:
base_model.compile(loss="mean_squared_error", optimizer="nadam", metrics=["mean_squared_error"])


In [20]:
model_path="../models/base_model"
early_stopings = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
checkpoint =  tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0)
callbacks=[early_stopings,checkpoint]

history = base_model.fit(X,pco2_images, epochs=100, validation_data=(X,pco2_images),workers=-1,batch_size=12,callbacks=callbacks)


NameError: name 'val_X' is not defined

In [ ]:
predicted_image=base_model.predict(val_X[419:421],verbose=1)

In [ ]:
plot_image(np.squeeze(predicted_image[1]))

In [ ]:
difference=np.squeeze(val_pco2_images[419:421][1])-np.squeeze(predicted_image[0])
plot_image(difference)

In [ ]:
plot_image(np.squeeze(val_pco2_images[419:421][1]))